# Greeks

- オプションの感応度を示す
- ブラックショールズモデルでリスクを管理する場合に重要
- jquants-derivativesの `Option` クラスではGreeksを自動で算出
- jquants-derivativesの `bsm` モジュールの関数から算出できる

In [1]:
from jquants_derivatives import Client, Option, bsm

option = Option(Client().get_option_index_option("2023-10-11"))
contract_month = option.contract_month[0]
greeks = (
    option.contracts_dfs[contract_month]
    .loc[
        :,
        [
            "ContractMonth",
            "PutCallDivision",
            "StrikePrice",
            "ImpliedVolatility",
            "Delta",
            "Gamma",
            "Vega",
            "Theta",
        ],
    ]
    .set_index("StrikePrice")
)
greeks

,ContractMonth,PutCallDivision,ImpliedVolatility,Delta,Gamma,Vega,Theta
StrikePrice,,,,,,,
29875.0,2023-10,1,0.350805,-0.000147,9.692271e-07,0.950109,-60.829758
30000.0,2023-10,1,0.348172,-0.000315,1.987275e-06,1.933454,-122.858584
30125.0,2023-10,1,0.338429,-0.000516,3.233426e-06,3.057825,-188.868239
30250.0,2023-10,1,0.323312,-0.000712,4.558950e-06,4.118781,-243.035908
30375.0,2023-10,1,0.314012,-0.001212,7.648356e-06,6.711141,-384.612871
30500.0,2023-10,1,0.301499,-0.001879,1.188841e-05,10.015938,-551.137825
30625.0,2023-10,1,0.293883,-0.003399,2.084054e-05,17.114551,-917.961381
30750.0,2023-10,1,0.281867,-0.005461,3.319295e-05,26.144003,-1344.939733
30875.0,2023-10,1,0.266672,-0.008225,5.038669e-05,37.547001,-1827.434184


In [2]:
s = option.underlying_price[contract_month]
t = option.time_to_maturity[contract_month]
r = option.interest_rate[contract_month] * 0.01
print(f"{s=}, {t=}, {r=}")

s=31936.51, t=0.0027397260273972603, r=-0.00041799999999999997


In [3]:
k = 32125
sigma = greeks.loc[k, "ImpliedVolatility"]
print(f"{k=}, {sigma=}")

k=32125, sigma=0.185425


## Delta

$$\Delta = \frac{\partial V}{\partial S}$$

- 原資産価格の変動に対する感度
- Delta=0.5のコールオプションで、原資産価格が100上昇すると、オプション価格は50上昇する

In [4]:
bsm.delta_call(s, k, t, r, sigma)

0.2596148409950075

In [5]:
bsm.delta_put(s, k, t, r, sigma)

-0.7403851590049926

### Gamma

$$\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}$$

- 原資産価格の変動に対するDeltaへの感度
- Gamma=0.0002のオプションで、原資産価格が100上昇すると、デルタは0.02上昇する

In [7]:
bsm.gamma(s, k, t, r, sigma)

0.001045665462371926

### Vega

$$\mathcal{V}=\frac{\partial V}{\partial \sigma}$$

-  ボラティリティの変動に対する感度
- Vega=2000のオプションで、ボラティリティが0.01(例: 14%から15%)上昇すると、オプション価格は20上昇する

In [8]:
bsm.vega(s, k, t, r, sigma)

541.8051103409326

### Theta

$$\Theta = -\frac{\partial V}{\partial \tau}$$

-  時間の経過に対する感度
- Theta=-3000のオプションで、1日（1/365）経過すると、オプション価格は8.2下落する

In [8]:
bsm.theta_call(s, k, t, r, sigma)

-18331.27472920579

In [9]:
bsm.theta_put(s, k, t, r, sigma)

-18344.702994583902